<a href="https://colab.research.google.com/github/Rryyuuu/capstone/blob/main/yolo_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir /content/dataset
!tar -xf '/content/drive/MyDrive/대학원/건물탐지 데이터셋/AIcrowd/train.tar' -C '/content/dataset'
!tar -xf '/content/drive/MyDrive/대학원/건물탐지 데이터셋/AIcrowd/val.tar' -C '/content/dataset'

mkdir: cannot create directory ‘/content/dataset’: File exists


In [9]:
import json
import os

# 경로 설정: COCO 주석 JSON 파일 경로와 출력 레이블 폴더 경로
coco_annotation_path = "/content/dataset/train/annotation.json"  # 실제 파일명에 맞게 수정
output_label_dir = "/content/dataset/train/labels"

os.makedirs(output_label_dir, exist_ok=True)  # 출력 폴더 생성

# COCO 주석 JSON 파일 로드
with open(coco_annotation_path, 'r') as f:
    coco_data = json.load(f)

images_info = coco_data["images"]
annotations = coco_data["annotations"]
categories = coco_data["categories"]

# 클래스 ID -> YOLO 클래스 인덱스 매핑 딕셔너리 생성
# (COCO의 category_id는 100 등 임의일 수 있으므로 0부터 연속된 index로 변환)
category_id_to_index = {}
for idx, cat in enumerate(categories):
    category_id_to_index[cat["id"]] = idx

# 이미지 ID별로 해당하는 annotations 모으기 (검색 속도 향상 위해 딕셔너리 사용)
anns_by_image = {}
for ann in annotations:
    img_id = ann["image_id"]
    if img_id not in anns_by_image:
        anns_by_image[img_id] = []
    anns_by_image[img_id].append(ann)

# 각 이미지에 대해 YOLO 라벨 파일 생성
for img in images_info:
    img_id = img["id"]
    file_name = img["file_name"]
    img_width = img["width"]
    img_height = img["height"]

    # 출력 파일 경로 (.txt), 이미지 파일명에서 확장자 제거 후 txt 붙임
    base_name, _ = os.path.splitext(file_name)
    label_file_path = os.path.join(output_label_dir, base_name + ".txt")

    # 해당 이미지의 모든 객체 주석 처리
    if img_id in anns_by_image:
        lines = []
        for ann in anns_by_image[img_id]:
            cat_id = ann["category_id"]
            # YOLO 클래스 인덱스
            class_idx = category_id_to_index[cat_id]

            # COCO bbox: [x_min, y_min, width, height]
            x_min, y_min, bw, bh = ann["bbox"]
            x_max = x_min + bw
            y_max = y_min + bh

            # 바운딩박스 중심좌표 및 폭/높이 계산
            x_center = (x_min + x_max) / 2.0
            y_center = (y_min + y_max) / 2.0
            # 정규화 (0~1 사이 값)
            x_center_norm = x_center / img_width
            y_center_norm = y_center / img_height
            w_norm = bw / img_width
            h_norm = bh / img_height

            # YOLO 형식 한 줄 구성: class_index x_center_norm y_center_norm w_norm h_norm
            line = f"{class_idx} {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
            lines.append(line)
        # 파일에 기록 (객체가 여러 개면 여러 줄 작성)
        with open(label_file_path, 'w') as lf:
            lf.write("\n".join(lines))
    else:
        # 이미지에 객체가 하나도 없는 경우 빈 파일 생성 (또는 생성 생략 가능)
        open(label_file_path, 'w').close()


In [10]:
import json
import os

# 경로 설정: COCO 주석 JSON 파일 경로와 출력 레이블 폴더 경로
coco_annotation_path = "/content/dataset/val/annotation.json"  # 실제 파일명에 맞게 수정
output_label_dir = "/content/dataset/val/labels"

os.makedirs(output_label_dir, exist_ok=True)  # 출력 폴더 생성

# COCO 주석 JSON 파일 로드
with open(coco_annotation_path, 'r') as f:
    coco_data = json.load(f)

images_info = coco_data["images"]
annotations = coco_data["annotations"]
categories = coco_data["categories"]

# 클래스 ID -> YOLO 클래스 인덱스 매핑 딕셔너리 생성
# (COCO의 category_id는 100 등 임의일 수 있으므로 0부터 연속된 index로 변환)
category_id_to_index = {}
for idx, cat in enumerate(categories):
    category_id_to_index[cat["id"]] = idx

# 이미지 ID별로 해당하는 annotations 모으기 (검색 속도 향상 위해 딕셔너리 사용)
anns_by_image = {}
for ann in annotations:
    img_id = ann["image_id"]
    if img_id not in anns_by_image:
        anns_by_image[img_id] = []
    anns_by_image[img_id].append(ann)

# 각 이미지에 대해 YOLO 라벨 파일 생성
for img in images_info:
    img_id = img["id"]
    file_name = img["file_name"]
    img_width = img["width"]
    img_height = img["height"]

    # 출력 파일 경로 (.txt), 이미지 파일명에서 확장자 제거 후 txt 붙임
    base_name, _ = os.path.splitext(file_name)
    label_file_path = os.path.join(output_label_dir, base_name + ".txt")

    # 해당 이미지의 모든 객체 주석 처리
    if img_id in anns_by_image:
        lines = []
        for ann in anns_by_image[img_id]:
            cat_id = ann["category_id"]
            # YOLO 클래스 인덱스
            class_idx = category_id_to_index[cat_id]

            # COCO bbox: [x_min, y_min, width, height]
            x_min, y_min, bw, bh = ann["bbox"]
            x_max = x_min + bw
            y_max = y_min + bh

            # 바운딩박스 중심좌표 및 폭/높이 계산
            x_center = (x_min + x_max) / 2.0
            y_center = (y_min + y_max) / 2.0
            # 정규화 (0~1 사이 값)
            x_center_norm = x_center / img_width
            y_center_norm = y_center / img_height
            w_norm = bw / img_width
            h_norm = bh / img_height

            # YOLO 형식 한 줄 구성: class_index x_center_norm y_center_norm w_norm h_norm
            line = f"{class_idx} {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
            lines.append(line)
        # 파일에 기록 (객체가 여러 개면 여러 줄 작성)
        with open(label_file_path, 'w') as lf:
            lf.write("\n".join(lines))
    else:
        # 이미지에 객체가 하나도 없는 경우 빈 파일 생성 (또는 생성 생략 가능)
        open(label_file_path, 'w').close()

In [14]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [16]:
from ultralytics import YOLO
import yaml

# 데이터셋 YAML 구성
data_yaml = {
    'path': '/content/dataset',          # dataset 폴더 경로
    'train': 'train',
    'val': 'val',
    'names': ['building']               # 클래스 이름
}
with open('mapping_data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

# YOLOv8n 모델로 테스트 훈련
model = YOLO('yolov8m.pt')  # 사전학습 가중치 로드
model.train(data='mapping_data.yaml', epochs=1, imgsz=320, batch=64)


100%|██████████| 49.7M/49.7M [00:00<00:00, 407MB/s]


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=mapping_data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 123MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 389MB/s]


...

        1/1      2.25G      1.812      1.925      1.752         55        320: 100%|██████████| 17547/17547 [48:36<00:00,  6.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1885/1885 [07:37<00:00,  4.12it/s]


                   all      60317     515280      0.435      0.556      0.367      0.195

1 epochs completed in 0.946 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1885/1885 [07:00<00:00,  4.48it/s]


                   all      60317     515280      0.434      0.557      0.367      0.195
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x77fbeb1f9090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 